In [1]:
from src.aco.ACOSolver import ACOSolver
import pandas as pd
import numpy as np
import os
%matplotlib inline

In [2]:
rng = np.random.default_rng(42)  # Initialise a random number generator

## Porównanie czasów rozwiązań
Średnie i odchylenia standardowe czasów rozwiązań przykładowych Sudoku

In [3]:
NUM_RUNS = 10

In [4]:
time_results = pd.DataFrame()

In [5]:
BOARD_PATHS = [f"../resources/boards/logic-solvable/{d}/{f}" for d in os.listdir('../resources/boards/logic-solvable') for f in os.listdir(f"../resources/boards/logic-solvable/{d}")]


In [6]:
for board_path in BOARD_PATHS:
    times = []
    iterations = []
    num_failed = 0
    print(f"Solving {board_path.split('/')[-1]} ...")
    for i in range(NUM_RUNS):
        solver = ACOSolver(
                random_generator=rng,
                board_size=9,
                board_file=board_path,
                num_ants=10,
                max_iterations=1000,
                greediness=0.9,
                pheromone_decay=0.1,
                evaporation_rate=0.9,
                best_evaporation_rate=0.005
                )
        is_solved, all_fixed_values_best, all_pheromones_to_add, all_perc_fixed_cells, all_iterations = solver.solve(logging=False)
        times.append(solver.solution_time)
        iterations.append(len(all_iterations))
        pd.DataFrame({'fixed_values_best': all_fixed_values_best, 'pheromones_to_add': all_pheromones_to_add, 'perc_fixed_cells': all_perc_fixed_cells, 'iterations': all_iterations}).to_csv(path_or_buf=f"aco_experiment_results/{board_path.split('/')[-2]}_{board_path.split('/')[-1][:-4]}_{i}.csv", index=False)
        if not is_solved:
            num_failed += 1
    time_results = pd.concat([time_results, pd.DataFrame({
            'level': [board_path.split('/')[-2]],
            'board_name': [board_path.split('/')[-1]],
            'time_mean': [np.mean(times)],
            'time_std': [np.std(times)],
            'time_min': [np.min(times)],
            'time_max': [np.max(times)],
            'iter_mean': [np.mean(iterations)],
            'iter_std': [np.std(iterations)],
            'iter_min': [np.min(iterations)],
            'iter_max': [np.max(iterations)],
            'num_failed': num_failed
            })], ignore_index=True, axis=0)
    print(f"Solved {board_path.split('/')[-1]}")

Solving board2.txt ...
Solved board2.txt
Solving platiniumblonde.txt ...
Solved platiniumblonde.txt
Solving aiescargot.txt ...
Solved aiescargot.txt
Solving board1.txt ...
Solved board1.txt
Solving coly013.txt ...
Solved coly013.txt
Solving reddwarf.txt ...
Solved reddwarf.txt
Solving sabuncu1.txt ...
Solved sabuncu1.txt
Solving board3.txt ...
Solved board3.txt
Solving goldennugget.txt ...
Solved goldennugget.txt
Solving board2.txt ...
Solved board2.txt
Solving board1.txt ...
Solved board1.txt
Solving board3.txt ...
Solved board3.txt
Solving board2.txt ...
Solved board2.txt
Solving board1.txt ...
Solved board1.txt
Solving board3.txt ...
Solved board3.txt


In [7]:
time_results.to_csv(path_or_buf='aco_experiment_results/time_results.csv', index=False)

In [8]:
time_results

,level,board_name,time_mean,time_std,time_min,time_max,iter_mean,iter_std,iter_min,iter_max,num_failed
0,hard,board2.txt,0.008206,0.000384,0.007944,0.009142,1.0,0.000000,1,1,0
1,hard,platiniumblonde.txt,11.392835,9.606477,0.901811,30.958899,74.0,62.358640,6,202,0
2,hard,aiescargot.txt,2.422455,1.910435,0.289865,6.291389,16.7,13.069430,2,43,0
3,hard,board1.txt,0.068225,0.045015,0.007936,0.108428,1.0,0.000000,1,1,0
4,hard,coly013.txt,58.260160,28.849048,8.073121,112.798814,372.8,183.554787,51,720,0
5,hard,reddwarf.txt,7.617370,6.997312,0.150588,21.908373,49.9,45.874721,1,144,0
6,hard,sabuncu1.txt,0.007998,0.000046,0.007914,0.008067,1.0,0.000000,1,1,0
7,hard,board3.txt,0.140412,0.037680,0.125938,0.253405,1.1,0.300000,1,2,0
8,hard,goldennugget.txt,8.608578,8.630374,0.450451,31.567841,57.1,57.390679,3,210,0
9,medium,board2.txt,0.007609,0.000183,0.007517,0.008147,1.0,0.000000,1,1,0


## Porównanie hiperparametrów
Dokonam porównania wpływu hiperparametrów na uśrednionych wynikach rozwiązując Sudoku **aiescargot**

#### Liczba mrówek

In [9]:
NUM_ANTS = [1, 5, 10, 20, 50, 100]
BOARD_PATH = f"../resources/boards/logic-solvable/hard/aiescargot.txt"

In [10]:
ants_results = pd.DataFrame()

In [11]:
for num_ants in NUM_ANTS:
    times = []
    iterations = []
    num_failed = 0
    print(f"Solving for {num_ants} ants ...")
    for i in range(NUM_RUNS):
        solver = ACOSolver(
                random_generator=rng,
                board_size=9,
                board_file=BOARD_PATH,
                num_ants=num_ants,
                max_iterations=1000,
                greediness=0.9,
                pheromone_decay=0.1,
                evaporation_rate=0.9,
                best_evaporation_rate=0.005
                )
        is_solved, all_fixed_values_best, all_pheromones_to_add, all_perc_fixed_cells, all_iterations = solver.solve(logging=False)
        times.append(solver.solution_time)
        iterations.append(len(all_iterations))
        if not is_solved:
            num_failed += 1
    ants_results = pd.concat([ants_results, pd.DataFrame({
            'board_name': BOARD_PATH.split('/')[-1],
            'num_ants': num_ants,
            'greediness': 0.9,
            'pheromone_decay': 0.1,
            'evaporation_rate': 0.9,
            'best_evaporation_rate': 0.005,
            'time_mean': [np.mean(times)],
            'time_std': [np.std(times)],
            'time_min': [np.min(times)],
            'time_max': [np.max(times)],
            'iter_mean': [np.mean(iterations)],
            'iter_std': [np.std(iterations)],
            'iter_min': [np.min(iterations)],
            'iter_max': [np.max(iterations)],
            'num_failed': num_failed
            })], ignore_index=True, axis=0)
    print(f"Solved for {num_ants} ants")

Solving for 1 ants ...
Solved for 1 ants
Solving for 5 ants ...
Solved for 5 ants
Solving for 10 ants ...
Solved for 10 ants
Solving for 20 ants ...
Solved for 20 ants
Solving for 50 ants ...
Solved for 50 ants
Solving for 100 ants ...
Solved for 100 ants


In [12]:
ants_results

,board_name,num_ants,greediness,pheromone_decay,evaporation_rate,best_evaporation_rate,time_mean,time_std,time_min,time_max,iter_mean,iter_std,iter_min,iter_max,num_failed
0,aiescargot.txt,1,0.9,0.1,0.9,0.005,2.802816,3.355229,0.275145,10.587012,193.4,233.714013,19,740,0
1,aiescargot.txt,5,0.9,0.1,0.9,0.005,1.574564,1.758114,0.139180,6.605092,21.5,23.576471,2,89,0
2,aiescargot.txt,10,0.9,0.1,0.9,0.005,1.779234,1.297786,0.561314,4.661442,12.6,9.156419,4,33,0
3,aiescargot.txt,20,0.9,0.1,0.9,0.005,2.214289,1.755973,0.281245,5.086488,7.8,6.209670,1,18,0
4,aiescargot.txt,50,0.9,0.1,0.9,0.005,4.327330,2.570515,0.693930,8.651706,6.1,3.562303,1,12,0
5,aiescargot.txt,100,0.9,0.1,0.9,0.005,3.941519,2.874999,1.362593,11.267504,2.8,2.039608,1,8,0


In [13]:
ants_results.to_csv(path_or_buf='aco_experiment_results/ants_results.csv', index=False)

#### Zachłanność algorytmu

In [14]:
GREEDINESS = [0, 0.2, 0.4, 0.6, 0.8, 1.0]

In [15]:
greediness_results = pd.DataFrame()

In [16]:
for greediness in GREEDINESS:
    times = []
    iterations = []
    num_failed = 0
    print(f"Solving for greediness={greediness} ...")
    for i in range(NUM_RUNS):
        solver = ACOSolver(
                random_generator=rng,
                board_size=9,
                board_file=BOARD_PATH,
                num_ants=10,
                max_iterations=1000,
                greediness=greediness,
                pheromone_decay=0.1,
                evaporation_rate=0.9,
                best_evaporation_rate=0.005
                )
        is_solved, all_fixed_values_best, all_pheromones_to_add, all_perc_fixed_cells, all_iterations = solver.solve(logging=False)
        times.append(solver.solution_time)
        iterations.append(len(all_iterations))
        if not is_solved:
            num_failed += 1
    greediness_results = pd.concat([greediness_results, pd.DataFrame({
            'board_name': BOARD_PATH.split('/')[-1],
            'num_ants': 10,
            'greediness': greediness,
            'pheromone_decay': 0.1,
            'evaporation_rate': 0.9,
            'best_evaporation_rate': 0.005,
            'time_mean': [np.mean(times)],
            'time_std': [np.std(times)],
            'time_min': [np.min(times)],
            'time_max': [np.max(times)],
            'iter_mean': [np.mean(iterations)],
            'iter_std': [np.std(iterations)],
            'iter_min': [np.min(iterations)],
            'iter_max': [np.max(iterations)],
            'num_failed': num_failed
            })], ignore_index=True, axis=0)
    print(f"Solved for greediness={greediness}")

Solving for greediness=0 ...
Solved for greediness=0
Solving for greediness=0.2 ...
Solved for greediness=0.2
Solving for greediness=0.4 ...
Solved for greediness=0.4
Solving for greediness=0.6 ...
Solved for greediness=0.6
Solving for greediness=0.8 ...
Solved for greediness=0.8
Solving for greediness=1.0 ...
Solved for greediness=1.0


In [17]:
greediness_results

,board_name,num_ants,greediness,pheromone_decay,evaporation_rate,best_evaporation_rate,time_mean,time_std,time_min,time_max,iter_mean,iter_std,iter_min,iter_max,num_failed
0,aiescargot.txt,10,0.0,0.1,0.9,0.005,6.512431,7.431533,0.278663,21.840767,44.2,49.961585,2,145,0
1,aiescargot.txt,10,0.2,0.1,0.9,0.005,5.197916,4.379149,0.273371,13.080888,36.2,30.386839,2,92,0
2,aiescargot.txt,10,0.4,0.1,0.9,0.005,5.810130,6.988419,0.282536,23.140760,40.6,48.197925,2,159,0
3,aiescargot.txt,10,0.6,0.1,0.9,0.005,1.456209,1.243279,0.279592,4.547432,10.2,8.749857,2,32,0
4,aiescargot.txt,10,0.8,0.1,0.9,0.005,2.565303,1.748344,0.421815,4.965889,17.9,12.144546,3,34,0
5,aiescargot.txt,10,1.0,0.1,0.9,0.005,1.818015,1.613379,0.137778,5.312455,13.0,11.523888,1,38,0


In [18]:
greediness_results.to_csv(path_or_buf='aco_experiment_results/greediness_results.csv', index=False)

#### Rozpad feromonu

In [19]:
PHEROMONE_DECAYS = [0, 0.2, 0.4, 0.6, 0.8, 1.0]

In [20]:
pheromone_decay_results = pd.DataFrame()

In [21]:
for pheromone_decay in PHEROMONE_DECAYS:
    times = []
    iterations = []
    num_failed = 0
    print(f"Solving for pheromone_decay={pheromone_decay} ...")
    for i in range(NUM_RUNS):
        solver = ACOSolver(
                random_generator=rng,
                board_size=9,
                board_file=BOARD_PATH,
                num_ants=10,
                max_iterations=1000,
                greediness=0.9,
                pheromone_decay=pheromone_decay,
                evaporation_rate=0.9,
                best_evaporation_rate=0.005
                )
        is_solved, all_fixed_values_best, all_pheromones_to_add, all_perc_fixed_cells, all_iterations = solver.solve(logging=False)
        times.append(solver.solution_time)
        iterations.append(len(all_iterations))
        if not is_solved:
            num_failed += 1
    pheromone_decay_results = pd.concat([pheromone_decay_results, pd.DataFrame({
            'board_name': BOARD_PATH.split('/')[-1],
            'num_ants': 10,
            'greediness': 0.9,
            'pheromone_decay': pheromone_decay,
            'evaporation_rate': 0.9,
            'best_evaporation_rate': 0.005,
            'time_mean': [np.mean(times)],
            'time_std': [np.std(times)],
            'time_min': [np.min(times)],
            'time_max': [np.max(times)],
            'iter_mean': [np.mean(iterations)],
            'iter_std': [np.std(iterations)],
            'iter_min': [np.min(iterations)],
            'iter_max': [np.max(iterations)],
            'num_failed': num_failed
            })], ignore_index=True, axis=0)
    print(f"Solved for pheromone_decay={pheromone_decay}")

Solving for pheromone_decay=0 ...
Solved for pheromone_decay=0
Solving for pheromone_decay=0.2 ...
Solved for pheromone_decay=0.2
Solving for pheromone_decay=0.4 ...
Solved for pheromone_decay=0.4
Solving for pheromone_decay=0.6 ...
Solved for pheromone_decay=0.6
Solving for pheromone_decay=0.8 ...
Solved for pheromone_decay=0.8
Solving for pheromone_decay=1.0 ...
Solved for pheromone_decay=1.0


In [22]:
pheromone_decay_results

,board_name,num_ants,greediness,pheromone_decay,evaporation_rate,best_evaporation_rate,time_mean,time_std,time_min,time_max,iter_mean,iter_std,iter_min,iter_max,num_failed
0,aiescargot.txt,10,0.9,0.0,0.9,0.005,2.280355,2.213385,0.292112,7.802432,15.9,15.636176,2,55,0
1,aiescargot.txt,10,0.9,0.2,0.9,0.005,1.325873,1.666409,0.145116,5.728878,9.3,11.883181,1,41,0
2,aiescargot.txt,10,0.9,0.4,0.9,0.005,2.928828,2.071420,0.315489,6.235695,20.4,14.381933,2,43,0
3,aiescargot.txt,10,0.9,0.6,0.9,0.005,2.376467,1.870331,0.288360,5.986154,16.3,12.798828,2,41,0
4,aiescargot.txt,10,0.9,0.8,0.9,0.005,2.350599,2.762634,0.291231,9.003755,16.1,18.986574,2,62,0
5,aiescargot.txt,10,0.9,1.0,0.9,0.005,2.614125,2.287760,0.717631,7.965738,18.2,15.746746,5,54,0


In [23]:
pheromone_decay_results.to_csv(path_or_buf='aco_experiment_results/pheromone_decay_results.csv', index=False)

#### Parowanie feromonu

In [24]:
EVAPORATION_RATES = [0, 0.2, 0.4, 0.6, 0.8, 1.0]

In [25]:
evaporation_rates_results = pd.DataFrame()

In [26]:
for evaporation_rate in EVAPORATION_RATES:
    times = []
    iterations = []
    num_failed = 0
    print(f"Solving for evaporation_rate={evaporation_rate} ...")
    for i in range(NUM_RUNS):
        solver = ACOSolver(
                random_generator=rng,
                board_size=9,
                board_file=BOARD_PATH,
                num_ants=10,
                max_iterations=1000,
                greediness=0.9,
                pheromone_decay=0.1,
                evaporation_rate=evaporation_rate,
                best_evaporation_rate=0.005
                )
        is_solved, all_fixed_values_best, all_pheromones_to_add, all_perc_fixed_cells, all_iterations = solver.solve(logging=False)
        times.append(solver.solution_time)
        iterations.append(len(all_iterations))
        if not is_solved:
            num_failed += 1
    evaporation_rates_results = pd.concat([evaporation_rates_results, pd.DataFrame({
            'board_name': BOARD_PATH.split('/')[-1],
            'num_ants': 10,
            'greediness': 0.9,
            'pheromone_decay': 0.1,
            'evaporation_rate': evaporation_rate,
            'best_evaporation_rate': 0.005,
            'time_mean': [np.mean(times)],
            'time_std': [np.std(times)],
            'time_min': [np.min(times)],
            'time_max': [np.max(times)],
            'iter_mean': [np.mean(iterations)],
            'iter_std': [np.std(iterations)],
            'iter_min': [np.min(iterations)],
            'iter_max': [np.max(iterations)],
            'num_failed': num_failed
            })], ignore_index=True, axis=0)
    print(f"Solved for evaporation_rate={evaporation_rate}")

Solving for evaporation_rate=0 ...
Solved for evaporation_rate=0
Solving for evaporation_rate=0.2 ...
Solved for evaporation_rate=0.2
Solving for evaporation_rate=0.4 ...
Solved for evaporation_rate=0.4
Solving for evaporation_rate=0.6 ...
Solved for evaporation_rate=0.6
Solving for evaporation_rate=0.8 ...
Solved for evaporation_rate=0.8
Solving for evaporation_rate=1.0 ...
Solved for evaporation_rate=1.0


In [27]:
evaporation_rates_results

,board_name,num_ants,greediness,pheromone_decay,evaporation_rate,best_evaporation_rate,time_mean,time_std,time_min,time_max,iter_mean,iter_std,iter_min,iter_max,num_failed
0,aiescargot.txt,10,0.9,0.1,0.0,0.005,1.902497,1.736025,0.414757,6.316786,13.4,12.387090,3,45,0
1,aiescargot.txt,10,0.9,0.1,0.2,0.005,4.006092,3.541090,0.418752,11.652757,28.3,24.968180,3,82,0
2,aiescargot.txt,10,0.9,0.1,0.4,0.005,3.403578,1.925552,0.553886,7.171685,24.3,13.667845,4,51,0
3,aiescargot.txt,10,0.9,0.1,0.6,0.005,2.091734,1.204157,0.547697,4.159895,15.1,8.676981,4,30,0
4,aiescargot.txt,10,0.9,0.1,0.8,0.005,3.643099,5.312983,0.144367,18.880373,25.7,37.124251,1,132,0
5,aiescargot.txt,10,0.9,0.1,1.0,0.005,3.324364,2.503018,0.871765,9.329134,22.8,17.145262,6,64,0


In [28]:
evaporation_rates_results.to_csv(path_or_buf='aco_experiment_results/evaporation_rates_results.csv', index=False)

#### Parowanie feromonu najlepszej wartości

In [29]:
BEST_EVAPORATION_RATES = [0.0, 0.001, 0.01, 0.1, 1.0]

In [30]:
best_evaporation_rates_results = pd.DataFrame()

In [31]:
for best_evaporation_rate in BEST_EVAPORATION_RATES:
    times = []
    iterations = []
    num_failed = 0
    print(f"Solving for best_evaporation_rate={best_evaporation_rate} ...")
    for i in range(NUM_RUNS):
        solver = ACOSolver(
                random_generator=rng,
                board_size=9,
                board_file=BOARD_PATH,
                num_ants=10,
                max_iterations=1000,
                greediness=0.9,
                pheromone_decay=0.1,
                evaporation_rate=0.9,
                best_evaporation_rate=best_evaporation_rate
                )
        is_solved, all_fixed_values_best, all_pheromones_to_add, all_perc_fixed_cells, all_iterations = solver.solve(logging=False)
        times.append(solver.solution_time)
        iterations.append(len(all_iterations))
        if not is_solved:
            num_failed += 1
    best_evaporation_rates_results = pd.concat([best_evaporation_rates_results, pd.DataFrame({
            'board_name': BOARD_PATH.split('/')[-1],
            'num_ants': 10,
            'greediness': 0.9,
            'pheromone_decay': 0.1,
            'evaporation_rate': 0.9,
            'best_evaporation_rate': best_evaporation_rate,
            'time_mean': [np.mean(times)],
            'time_std': [np.std(times)],
            'time_min': [np.min(times)],
            'time_max': [np.max(times)],
            'iter_mean': [np.mean(iterations)],
            'iter_std': [np.std(iterations)],
            'iter_min': [np.min(iterations)],
            'iter_max': [np.max(iterations)],
            'num_failed': num_failed
            })], ignore_index=True, axis=0)
    print(f"Solved for best_evaporation_rate={best_evaporation_rate}")

Solving for best_evaporation_rate=0.0 ...
Solved for best_evaporation_rate=0.0
Solving for best_evaporation_rate=0.001 ...
Solved for best_evaporation_rate=0.001
Solving for best_evaporation_rate=0.01 ...
Solved for best_evaporation_rate=0.01
Solving for best_evaporation_rate=0.1 ...
Solved for best_evaporation_rate=0.1
Solving for best_evaporation_rate=1.0 ...
Solved for best_evaporation_rate=1.0


In [32]:
best_evaporation_rates_results

,board_name,num_ants,greediness,pheromone_decay,evaporation_rate,best_evaporation_rate,time_mean,time_std,time_min,time_max,iter_mean,iter_std,iter_min,iter_max,num_failed
0,aiescargot.txt,10,0.9,0.1,0.9,0.000,2.370763,1.668879,0.144860,4.895242,16.4,11.542963,1,34,0
1,aiescargot.txt,10,0.9,0.1,0.9,0.001,2.531556,2.017422,0.440653,7.585814,17.3,13.820637,3,52,0
2,aiescargot.txt,10,0.9,0.1,0.9,0.010,2.020039,1.541142,0.419530,4.921260,14.3,10.900000,3,35,0
3,aiescargot.txt,10,0.9,0.1,0.9,0.100,3.050178,2.659112,0.142730,9.921551,20.9,18.239792,1,68,0
4,aiescargot.txt,10,0.9,0.1,0.9,1.000,2.034090,1.194771,0.147021,4.400462,13.9,8.154140,1,30,0


In [33]:
best_evaporation_rates_results.to_csv(path_or_buf='aco_experiment_results/best_evaporation_rates_results.csv', index=False)